In [81]:
#Libraries and paths
#Setup as advised from http://alimanfoo.github.io/2016/06/10/scikit-allel-tour.html
import numpy as np
import sys
import scipy
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
sns.set_context('notebook')
import h5py
import zarr
import numcodecs
import allel; print('scikit-allel', allel.__version__)
from pybedtools import BedTool

project = "/faststorage/project/primatediversity/people/erik/"
vcf_dir = "/faststorage/project/primatediversity/data/variants/"
callability_dir = "/faststorage/project/primatediversity/data/callability/"
call_suffix = ".variable.filtered.callable.bed.gz"
vcf_suffix = ".variable.filtered.HF.snps.vcf.gz"
metainfo = "data/New_Papio.xlsx"

scikit-allel 1.3.2


In [26]:
#Setting up metainfo, and counting to verify that i did it right, fits with this list containing all new baboon samples.
read_meta = pd.read_excel(project+metainfo)
meta_dir = {}
count_dir = {}
for i, row in read_meta.iterrows():
    if row.Species not in meta_dir:
        meta_dir[row.Species] = [row.PGDP_ID]
        count_dir[row.Species] = 1
    else:
        meta_dir[row.Species].append(row.PGDP_ID)
        count_dir[row.Species] += 1

In [27]:
print(meta_dir)
print(count_dir)

{'gelada': ['PD_0067'], 'anubis': ['PD_0199', 'PD_0200', 'PD_0201', 'PD_0202', 'PD_0203', 'PD_0204', 'PD_0205', 'PD_0206', 'PD_0207', 'PD_0208', 'PD_0209', 'PD_0210', 'PD_0211', 'PD_0212', 'PD_0218', 'PD_0219', 'PD_0220', 'PD_0221', 'PD_0222', 'PD_0238', 'PD_0239', 'PD_0240', 'PD_0241', 'PD_0242', 'PD_0243', 'PD_0244', 'PD_0265', 'PD_0266', 'PD_0267', 'PD_0492', 'PD_0493', 'PD_0494', 'PD_0495', 'PD_0496', 'PD_0497', 'PD_0498', 'PD_0499', 'PD_0500', 'PD_0501', 'PD_0502', 'PD_0504', 'PD_0505', 'PD_0506', 'PD_0507', 'PD_0508', 'PD_0509', 'PD_0641', 'PD_0642', 'PD_0649', 'PD_0650', 'PD_0651', 'PD_0652', 'PD_0653', 'PD_0654', 'PD_0674', 'PD_0675', 'PD_0676', 'PD_0677', 'PD_0678', 'PD_0679', 'PD_0680', 'PD_0681', 'PD_0682', 'PD_0683', 'PD_0684', 'PD_0685', 'PD_0686', 'PD_0720', 'PD_0721', 'PD_0722', 'PD_0723', 'PD_0724', 'PD_0725', 'PD_0726', 'PD_0727', 'PD_0728', 'PD_0729', 'PD_0730', 'PD_0731', 'PD_0732', 'PD_0733', 'PD_0734', 'PD_0735', 'PD_0736', 'PD_0737', 'PD_0738', 'PD_0739', 'PD_0740

In [52]:
hamadryas = meta_dir["hamadryas"]
first_hamadryas = hamadryas[0]
callset_test = allel.read_vcf(vcf_dir+first_hamadryas+vcf_suffix, region = "chr20", fields=['variants/POS'])
callset_test

{'variants/POS': array([    2976,     2990,     2994, ..., 77137074, 77137113, 77137130],
       dtype=int32)}

In [55]:
len(callset_test["variants/POS"])

1201925

In [59]:
#The number of variant sites varies for each individual
for ind in hamadryas:
    print(ind)
    callset = allel.read_vcf(vcf_dir+ind+vcf_suffix, region = "chr20", fields=['variants/POS'])
    print(len(callset["variants/POS"]))

PD_0696
2631234
PD_0697
2638773
PD_0698
2632008
PD_0699
2654473
PD_0700
2651345
PD_0701
2645414
PD_0702
2646403
PD_0703
2640769
PD_0704
2603202
PD_0705
2643493
PD_0706
2637162
PD_0707
2642600
PD_0708
2630815
PD_0709
2641012
PD_0710
2613921
PD_0711
2630032
PD_0712
2645887
PD_0713
2635686
PD_0714
2620140
PD_0715
2626902
PD_0716
2634585
PD_0717
2648397
PD_0718
2645024
PD_0719
2638562


In [77]:
hamadryas = meta_dir["hamadryas"]
first_hamadryas = hamadryas[0]
chromosomes = list(range(20, 21))+["X"]
for chrom in chromosomes:
    positions = allel.read_vcf(vcf_dir+first_hamadryas+vcf_suffix, region = "chr{}".format(chrom), fields=['variants/POS'])["variants/POS"]
    poslist = []
    for pos in positions:
        poslist.append(pos)
    call_df = pd.DataFrame(data=poslist, columns = ["pos"])
    bed_call = BedTool(callability_dir+first_hamadryas+call_suffix)
    for interval in bed_call:
        

In [94]:
bed_call = BedTool(callability_dir+hamadryas[0]+call_suffix)
bed_call.head(1000)

chr1	930	936
 chr1	937	945
 chr1	946	961
 chr1	962	969
 chr1	972	986
 chr1	987	988
 chr1	989	990
 chr1	991	1005
 chr1	1006	1009
 chr1	1010	1038
 chr1	1039	1046
 chr1	1049	1062
 chr1	1065	1066
 chr1	1068	1080
 chr1	1081	1083
 chr1	1084	1087
 chr1	1088	1186
 chr1	4980	5070
 chr1	5071	5088
 chr1	5089	5130
 chr1	5148	5325
 chr1	5329	5335
 chr1	5336	5601
 chr1	5631	5781
 chr1	6186	6416
 chr1	6417	6432
 chr1	6433	6461
 chr1	6525	6584
 chr1	6970	7147
 chr1	10669	10835
 chr1	10837	10907
 chr1	10912	10916
 chr1	10917	10961
 chr1	10977	11032
 chr1	11756	11783
 chr1	11797	12821
 chr1	12823	12826
 chr1	12952	12974
 chr1	12977	12991
 chr1	13009	13011
 chr1	13389	13524
 chr1	14052	14168
 chr1	14269	14404
 chr1	14406	14933
 chr1	15051	15258
 chr1	15259	15272
 chr1	15273	15303
 chr1	15357	15758
 chr1	15766	16044
 chr1	16062	16077
 chr1	16136	16151
 chr1	16160	17293
 chr1	17303	17306
 chr1	17311	17312
 chr1	17315	17316
 chr1	17318	17320
 chr1	17323	17326
 chr1	17327	17330
 chr1	17331	17412
 chr1	17415	